#### 1. Environment Setup & Data Loading

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df_players = pd.read_csv('players.csv')
df_trans = pd.read_csv('transactions.csv')
df_affiliates = pd.read_csv('affiliates.csv')

# Convert date columns to datetime objects
df_players['reg_date'] = pd.to_datetime(df_players['reg_date'])
df_trans['trans_date'] = pd.to_datetime(df_trans['trans_date'])

print("Data Loaded Successfully.")
df_players.head()

Data Loaded Successfully.


,player_id,affiliate_id,reg_date,country
0,1000,105,2025-01-30,UK
1,1001,102,2025-01-12,IN
2,1002,102,2025-01-10,IN
3,1003,102,2025-01-26,NG
4,1004,105,2025-01-31,UK


In [2]:
df_trans.head()

,player_id,type,amount,trans_date
0,1000,Withdrawal,125.22,2025-01-17
1,1000,Bonus,431.43,2025-01-07
2,1000,Bonus,281.67,2025-01-14
3,1001,Deposit,338.95,2025-01-27
4,1001,Withdrawal,447.26,2025-01-23


In [3]:
df_affiliates.head()

,affiliate_id,affiliate_name,commission_model
0,101,Google_Ads,CPA
1,102,Twitch_Influencer,RevShare
2,103,SEO_Blog_Casino,CPA
3,104,Facebook_Alpha,Hybrid
4,105,Telegram_Signals,CPA


#### 2. Data Validation & Quality Check

##### Check for missing values and duplicates that could skew ROI calculations.

In [4]:
# Check for missing values
print("Missing values in Players:", df_players.isnull().sum().sum())
print("Missing values in Transactions:", df_trans.isnull().sum().sum())

# Check for duplicate transactions
duplicates = df_trans.duplicated().sum()
if duplicates > 0:
    df_trans = df_trans.drop_duplicates()
    print(f"Removed {duplicates} duplicate records.")
else:
    print("No duplicates found.")

Missing values in Players: 0
Missing values in Transactions: 0
No duplicates found.


#### 3. Feature Engineering (Calculating NGR)

##### Calculate Net Gaming Revenue (NGR), we must pivot the transaction types. NGR = Deposits - Withdrawals - Bonuses.

In [5]:
# Pivot transactions to get a single row per player
player_finance = df_trans.pivot_table(
    index='player_id', 
    columns='type', 
    values='amount', 
    aggfunc='sum'
).fillna(0)

# Calculate NGR
player_finance['NGR'] = player_finance['Deposit'] - player_finance['Withdrawal'] - player_finance['Bonus']

# Merge with player info to link to Affiliates
df_master = df_players.merge(player_finance, on='player_id', how='left').merge(df_affiliates, on='affiliate_id')

df_master.head()

,player_id,affiliate_id,reg_date,country,Bonus,Deposit,Withdrawal,NGR,affiliate_name,commission_model
0,1000,105,2025-01-30,UK,713.10,0.00,125.22,-838.32,Telegram_Signals,CPA
1,1001,102,2025-01-12,IN,834.21,338.95,1277.33,-1772.59,Twitch_Influencer,RevShare
2,1002,102,2025-01-10,IN,379.37,49.70,241.05,-570.72,Twitch_Influencer,RevShare
3,1003,102,2025-01-26,NG,330.94,321.18,887.36,-897.12,Twitch_Influencer,RevShare
4,1004,105,2025-01-31,UK,95.56,265.84,221.63,-51.35,Telegram_Signals,CPA


#### 4. Identifying "Bonus Hunters" (Anomaly Detection)

Identify affiliates bringing in 'Bonus Hunters' by calculating the Bonus-to-Deposit ratio. 
A ratio > 1.0 means the player is costing the company more in bonuses than they are depositing.

In [6]:
# Aggregate by Affiliate
affiliate_stats = df_master.groupby('affiliate_name').agg({
    'player_id': 'count',
    'Deposit': 'sum',
    'Bonus': 'sum',
    'NGR': 'sum'
}).rename(columns={'player_id': 'total_players'})

# Calculate Quality Metrics
affiliate_stats['bonus_ratio'] = affiliate_stats['Bonus'] / (affiliate_stats['Deposit'] + 1) # +1 to avoid div by zero
affiliate_stats['roi_percentage'] = (affiliate_stats['NGR'] / (affiliate_stats['Deposit'] + 1)) * 100

# Highlighting the "Toxic" affiliates
toxic_partners = affiliate_stats[affiliate_stats['bonus_ratio'] > 0.8].sort_values(by='bonus_ratio', ascending=False)
toxic_partners

,total_players,Deposit,Bonus,NGR,bonus_ratio,roi_percentage
affiliate_name,,,,,,
Bonus_Hunter_Site,160,2281.68,25302.06,-25083.21,11.084366,-1098.849160
Google_Ads,159,75803.84,78096.41,-73590.74,1.030230,-97.079210
SEO_Blog_Casino,163,68783.57,68865.55,-67888.33,1.001177,-98.697033
Twitch_Influencer,166,77754.88,74917.05,-72600.66,0.963490,-93.369993
Telegram_Signals,170,85221.20,76722.03,-70469.49,0.900259,-82.689123
Facebook_Alpha,182,93143.27,77582.78,-67383.61,0.832931,-72.343269


#### 5. Detecting Traffic Spikes (Z-Score)

##### Automated monitoring for bot traffic or suspicious registration spikes using Z-score

In [7]:
# Group registrations by date and affiliate
daily_reg = df_players.groupby(['reg_date', 'affiliate_id']).size().reset_index(name='reg_count')

# Calculate mean and std for the Z-score
stats = daily_reg.groupby('affiliate_id')['reg_count'].agg(['mean', 'std']).reset_index()
daily_reg = daily_reg.merge(stats, on='affiliate_id')

# Z-score formula: (x - mean) / std
daily_reg['z_score'] = (daily_reg['reg_count'] - daily_reg['mean']) / daily_reg['std'].replace(0, 1)

# Flag anomalies where Z-score > 2 (95th percentile)
anomalies = daily_reg[daily_reg['z_score'] > 2]

print("Detected Registration Spikes:")
anomalies

Detected Registration Spikes:


,reg_date,affiliate_id,reg_count,mean,std,z_score
53,2025-01-10,101,10,5.129032,2.140747,2.275359
54,2025-01-10,102,13,5.354839,2.402060,3.182752
58,2025-01-10,106,11,5.161290,2.310797,2.526708
87,2025-01-15,105,91,5.666667,16.174338,5.275847
167,2025-01-29,101,10,5.129032,2.140747,2.275359


#### 6. Data Export

##### The final step is exporting this structured data for visualization.

In [9]:
# Save the master table
df_master.to_csv('cleaned_affiliate_performance.csv', index=False)